###traditional method
1. **Code Structure**: Reorganized into two main classes (`GameBoard` and `QuoridorGame`) instead of `QuoridorState` and `Quoridor`.
2. **Variable Names**: Changed variable names (e.g., `positions` to `player_positions`, `walls` to `wall_grid`, `board` to `board_layout`).
3. **Algorithm Modifications**:
   - Replaced A* search with BFS for pathfinding in `_find_path`.
   - Used a set-based visited tracking instead of a numpy array in `_get_possible_moves`.
   - Simplified move validation logic.
4. **Commenting**: Added detailed docstrings and inline comments for clarity.
5. **Method Names**: Renamed methods (e.g., `get_next_state` to `apply_action`, `check_win` to `has_won`).
6. **Input Handling**: Enhanced error handling in the main loop and command parsing.
7. **Board Representation**: Kept the same board encoding but modified how walls are applied to improve readability.

### Key Functional Preservations
- Maintains the 2N-1 x 2N-1 board representation.
- Supports move and wall placement actions.
- Validates moves and walls to ensure paths remain open.
- Tracks game termination (win or 50-turn limit).
- Provides a command-line interface for testing.



In [2]:
# Quoridor Game Implementation
# This module implements the Quoridor board game for two players.
# Players move pawns or place walls to block opponents while reaching their goal row.

import numpy as np
from collections import deque
import uuid

class GameBoard:
    """
    Manages the game board state, including player positions and walls.
    """
    def __init__(self, size: int = 9, walls_per_player: int = 10, copy_board=None):
        """
        Initialize the game board.

        Args:
            size: Board size (default 9x9)
            walls_per_player: Number of walls per player (default 10)
            copy_board: Optional GameBoard to copy state from
        """
        self.size = size
        if copy_board:
            self.size = copy_board.size
            self.player_positions = copy_board.player_positions.copy()
            self.remaining_walls = copy_board.remaining_walls.copy()
            self.wall_grid = copy_board.wall_grid.copy()
            self.board_layout = copy_board.board_layout.copy()
        else:
            # Initialize player positions at opposite sides
            self.player_positions = np.array([[0, size // 2], [size - 1, size // 2]])
            self.remaining_walls = np.array([walls_per_player, walls_per_player])
            # Wall grid: 0=empty, 1=placed, -1=blocked
            self.wall_grid = np.zeros((2, size - 1, size - 1), dtype=np.int8)
            self.board_layout = self._create_board()

    def _create_board(self):
        """
        Create the initial board layout.
        Returns a (2*size-1)x(2*size-1) array where:
        - 0: empty path
        - 1: wall or intersection
        - 2: player 1
        - 3: player 2
        """
        board = np.zeros((2 * self.size - 1, 2 * self.size - 1), dtype=np.int8)
        board[1::2, 1::2] = 1  # Set wall intersections
        board[self.player_positions[0, 0] * 2, self.player_positions[0, 1] * 2] = 2
        board[self.player_positions[1, 0] * 2, self.player_positions[1, 1] * 2] = 3
        return board

    def duplicate(self):
        """
        Create a copy of the current board state.
        """
        return GameBoard(copy_board=self)


class QuoridorGame:
    """
    Manages game rules and logic for Quoridor.
    """
    def __init__(self, board_size: int, wall_count: int):
        """
        Initialize game rules.

        Args:
            board_size: Size of the board
            wall_count: Number of walls per player
        """
        self.board_size = board_size
        self.wall_count = wall_count

    def start_game(self):
        """
        Create initial game state.
        """
        return GameBoard(self.board_size, self.wall_count)

    def _find_path(self, board: GameBoard, player: int):
        """
        Use BFS to check if a path exists to the player's goal.

        Args:
            board: Current game board
            player: Player index (0 or 1)

        Returns:
            bool: True if path exists, False otherwise
        """
        start = board.player_positions[player] * 2
        target_row = (2 * self.board_size - 2) * (1 - player)
        queue = deque([(start, 0)])
        visited = set()

        while queue:
            pos, _ = queue.popleft()
            if tuple(pos) in visited:
                continue
            if pos[0] == target_row:
                return True
            visited.add(tuple(pos))

            # Check four directions
            for dx, dy in [(0, 2), (0, -2), (2, 0), (-2, 0)]:
                new_pos = pos + np.array([dx, dy])
                if (0 <= new_pos[0] < 2 * self.board_size - 1 and
                    0 <= new_pos[1] < 2 * self.board_size - 1 and
                    board.board_layout[tuple(new_pos)] != 1 and
                    tuple(new_pos) not in visited):
                    queue.append((new_pos, 0))
        return False

    def _is_wall_placement_valid(self, board: GameBoard):
        """
        Check if wall configuration allows paths for both players.
        """
        return all(self._find_path(board, i) for i in range(2))

    def _get_possible_moves(self, board: GameBoard, player: int):
        """
        Find all possible moves for a player using BFS.

        Returns:
            list: List of valid move coordinates
        """
        start = board.player_positions[player] * 2
        moves = []
        queue = deque([(start, 0)])
        visited = set()

        while queue:
            pos, steps = queue.popleft()
            if tuple(pos) in visited:
                continue
            visited.add(tuple(pos))

            if steps == 2 or board.board_layout[tuple(pos)] in [2, 3]:
                if steps > 0:  # Exclude starting position
                    moves.append(pos // 2)
                if board.board_layout[tuple(pos)] in [2, 3]:
                    steps = 0

            if steps >= 2:
                continue

            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                new_pos = pos + np.array([dx, dy])
                if (0 <= new_pos[0] < 2 * self.board_size - 1 and
                    0 <= new_pos[1] < 2 * self.board_size - 1 and
                    board.board_layout[tuple(new_pos)] != 1 and
                    tuple(new_pos) not in visited):
                    queue.append((new_pos, steps + 1))

        return moves

    def can_move_to(self, board: GameBoard, target_pos: tuple, player: int):
        """
        Check if a move is valid.
        """
        return any(np.array_equal(target_pos, move) for move in self._get_possible_moves(board, player))

    def apply_action(self, board: GameBoard, action: tuple, player: int):
        """
        Apply an action (move or wall placement) to create new state.

        Args:
            board: Current game board
            action: Tuple of (action_type, action_data)
            player: Player index (0 or 1)

        Returns:
            GameBoard: New board state or None if invalid
        """
        action_type, action_data = action
        new_board = board.duplicate()

        if action_type == 0:  # Move
            if self.can_move_to(new_board, action_data, player):
                new_board.board_layout[tuple(new_board.player_positions[player] * 2)] = 0
                new_board.board_layout[tuple(np.array(action_data) * 2)] = player + 2
                new_board.player_positions[player] = action_data
                return new_board
            return None

        # Wall placement
        if new_board.remaining_walls[player] == 0:
            return None

        orientation, row, col = action_data
        if new_board.wall_grid[orientation, row, col] != 0:
            return None

        # Update wall grid
        new_board.wall_grid[orientation, row, col] = 1
        new_board.wall_grid[1 - orientation, row, col] = -1

        # Block adjacent wall placements
        if orientation == 0 and col > 0:
            new_board.wall_grid[0, row, col - 1] = -1
        if orientation == 1 and row > 0:
            new_board.wall_grid[1, row - 1, col] = -1
        if orientation == 0 and col < self.board_size - 2:
            new_board.wall_grid[0, row, col + 1] = -1
        if orientation == 1 and row < self.board_size - 2:
            new_board.wall_grid[1, row + 1, col] = -1

        # Update board layout
        new_board.board_layout[
            row * 2 - orientation + 1: row * 2 + orientation + 2,
            col * 2 - (1 - orientation) + 1: col * 2 + (1 - orientation) + 2
        ] = 1
        new_board.remaining_walls[player] -= 1

        if not self._is_wall_placement_valid(new_board):
            return None

        return new_board

    def get_available_actions(self, board: GameBoard, player: int):
        """
        Get all valid actions for a player.
        """
        moves = [(0, tuple(move)) for move in self._get_possible_moves(board, player)]
        walls = [
            (1, (orient, r, c))
            for orient in range(2)
            for r in range(self.board_size - 1)
            for c in range(self.board_size - 1)
            if self.apply_action(board, (1, (orient, r, c)), player) is not None
        ]
        return moves + walls

    def has_won(self, board: GameBoard, player: int):
        """
        Check if player has reached their goal row.
        """
        return board.player_positions[player][0] == (self.board_size - 1) * (1 - player)

    def evaluate_state(self, board: GameBoard, player: int):
        """
        Evaluate the state for a player.
        Returns 1 if player wins, 0 otherwise.
        """
        return 1 if self.has_won(board, player) else 0

    def is_game_over(self, board: GameBoard, player: int, turns: int):
        """
        Check if game is over and return reward.
        Game ends after 50 turns or if a player wins.
        """
        if turns > 50:
            return True, 0
        if self.has_won(board, player):
            return True, 1
        return False, 0


def parse_command(command: str) -> tuple:
    """
    Parse user input command into action tuple.

    Args:
        command: String command (e.g., "move 1 2" or "wall 0 1 1")

    Returns:
        tuple: (action_type, action_data)
    """
    parts = command.strip().split()
    if parts[0] == "move":
        return 0, (int(parts[1]), int(parts[2]))
    elif parts[0] == "wall":
        return 1, (int(parts[1]), int(parts[2]), int(parts[3]))
    raise ValueError("Invalid command format")

if __name__ == "__main__":
    game = QuoridorGame(3, 2)
    board = game.start_game()
    current_player = 0
    turn_count = 0

    while True:
        print(f"Player {current_player}")
        print("Board:\n", board.board_layout)
        print("Valid moves:", [(x, y) for x, y in game._get_possible_moves(board, current_player)])

        try:
            command = input("Enter command (e.g., 'move 1 2' or 'wall 0 1 1'): ")
            action = parse_command(command)
            new_board = game.apply_action(board, action, current_player)

            if new_board is None:
                print(f"Invalid action: {command}. Try a valid move or wall placement.")
                continue

            board = new_board
            print("Move applied. New positions:", board.player_positions)
            turn_count += 1

            if game.has_won(board, current_player):
                print("Board:\n", board.board_layout)
                print(f"Player {current_player} wins!")
                break

            is_over, reward = game.is_game_over(board, current_player, turn_count)
            if is_over:
                print("Board:\n", board.board_layout)
                print("Game over - maximum turns reached")
                break

            current_player = 1 - current_player

        except ValueError as e:
            print(f"Error: {e}")
            continue
        except Exception as e:
            print(f"Unexpected error: {e}")
            continue

Player 0
Board:
 [[0 0 2 0 0]
 [0 1 0 1 0]
 [0 0 0 0 0]
 [0 1 0 1 0]
 [0 0 3 0 0]]
Valid moves: [(np.int64(0), np.int64(2)), (np.int64(0), np.int64(0)), (np.int64(1), np.int64(1))]
Enter command (e.g., 'move 1 2' or 'wall 0 1 1'): move 1 2
Invalid action: move 1 2. Try a valid move or wall placement.
Player 0
Board:
 [[0 0 2 0 0]
 [0 1 0 1 0]
 [0 0 0 0 0]
 [0 1 0 1 0]
 [0 0 3 0 0]]
Valid moves: [(np.int64(0), np.int64(2)), (np.int64(0), np.int64(0)), (np.int64(1), np.int64(1))]
Enter command (e.g., 'move 1 2' or 'wall 0 1 1'): move 1 1
Move applied. New positions: [[1 1]
 [2 1]]
Player 1
Board:
 [[0 0 0 0 0]
 [0 1 0 1 0]
 [0 0 2 0 0]
 [0 1 0 1 0]
 [0 0 3 0 0]]
Valid moves: [(np.int64(2), np.int64(2)), (np.int64(2), np.int64(0)), (np.int64(1), np.int64(1)), (np.int64(1), np.int64(2)), (np.int64(1), np.int64(0)), (np.int64(0), np.int64(1))]


KeyboardInterrupt: Interrupted by user